In [2]:
import pandas as pd
import random
import numpy as np
from geopy.geocoders import Nominatim
import re
from tqdm import tqdm
import time

In [3]:
def clean_address(address):
    try:
        seps = re.compile(r'( St)|( Rd)|( Ave)|( Ct)|( Blvd)|( Dr)|( Ln)|( Pl)|( Sq)', flags = re.IGNORECASE)
        clean_addr = list(filter(None, re.split(seps, address)))
        clean_addr = clean_addr[0] + clean_addr[1]
        geolocator = Nominatim(user_agent="208final")
        location = geolocator.geocode(clean_addr)
        return (location.latitude, location.longitude)
    except:
        return np.nan

In [25]:
files = ["Carmel", "Diplo", "Federal", "Firstclass", "Highclass", "Prestige"]
utf = ["Carmel", "Diplo", "Firstclass", "Highclass", "Prestige"]
cols = {}
cols['Carmel'] = ['Date', 'Time', 'PU_Adress']
cols['Diplo'] = ['Date', 'Time', 'PU_Address']
cols['Federal'] = ['Date', 'Time', 'PU_Address']
cols['Firstclass'] = ['DATE', 'TIME', 'PICK UP ADDRESS']
cols['Highclass'] = ['DATE', 'TIME', 'PU_Address']
cols['Prestige'] = ['DATE', 'TIME', 'PICK UP ADDRESS']

def get_df(carrier):
    filename = ""
    if carrier in utf:
        filename = "raw_data/" + carrier + "_utf.csv"
    else:    
        filename = "raw_data/" + carrier + ".csv"
    print(filename)
    n = sum(1 for line in open(filename)) - 1
    s = int(n / 1000)
    print(s)
    skip = sorted(random.sample(range(1, n+1), n-s))
    df = pd.read_csv(filename, skiprows=skip)
    df = df.rename(columns={cols[carrier][0]: "date", cols[carrier][1]: "time", cols[carrier][2]: "location"})[["date", "time", "location"]]
    df["date"] = pd.to_datetime(df["date"])
    df["time"] = pd.to_datetime(df["time"]).dt.time
    df["location"] = [clean_address(row["location"]) for index, row in tqdm(df.iterrows())]
    df.dropna(subset=["location"], inplace = True)
    df["carrier"] = carrier.lower()
    return df

In [6]:
filename = "raw_data/American.csv"
n = sum(1 for line in open(filename)) - 1
s = int(n / 1000)
print(s)
skip = sorted(random.sample(range(1, n+1), n-s))
df = pd.read_csv(filename, skiprows=skip)
df = df.rename(columns={'DATE': "date", 'TIME': "time", 'PICK UP ADDRESS': "location"})[["date", "time", "location"]]
df["date"] = pd.to_datetime(df["date"])
df["time"] = pd.to_datetime(df["time"]).dt.time
geolocator = Nominatim(user_agent="208final")
df["location"] = [clean_address(row["location"]) for index, row in tqdm(df.iterrows())]
df.dropna(subset=["location"], inplace = True)
df["carrier"] = "american"
big_df = df

91


91it [00:43,  2.09it/s]


In [7]:
print(big_df)

         date      time                                 location   carrier
1  2014-07-04  17:15:00                 (40.8161391, -73.904608)  american
2  2014-07-06  12:33:00                (40.8192885, -73.8997237)  american
3  2014-07-07  04:45:00                (40.8152388, -73.9160874)  american
4  2014-07-09  11:54:00  (40.81610072727273, -73.90402954545455)  american
5  2014-07-09  19:01:00                 (40.815306, -73.8987435)  american
..        ...       ...                                      ...       ...
84 2014-09-27  01:54:00         (40.817868000000004, -73.853939)  american
86 2014-09-27  17:29:00                (40.8111478, -73.9168744)  american
87 2014-09-27  21:30:00        (40.82492209375, -73.90495953125)  american
89 2014-09-28  05:39:00                (40.8134483, -73.9153797)  american
90 2014-09-30  07:22:00                (40.8120839, -73.9041639)  american

[81 rows x 4 columns]


In [28]:
for file in files:
    big_df = big_df.append(get_df(file))

raw_data/Carmel_utf.csv
256


256it [05:11,  1.22s/it]


raw_data/Diplo_utf.csv
98


98it [00:49,  1.99it/s]


raw_data/Federal.csv
0


...

166it [02:30,  1.10it/s]


raw_data/Highclass_utf.csv
151


151it [01:22,  1.84it/s]


raw_data/Prestige_utf.csv
320


320it [02:34,  2.07it/s]


In [29]:
print(big_df)

          date      time                                  location   carrier
1   2014-07-04  17:15:00                  (40.8161391, -73.904608)  american
2   2014-07-06  12:33:00                 (40.8192885, -73.8997237)  american
3   2014-07-07  04:45:00                 (40.8152388, -73.9160874)  american
4   2014-07-09  11:54:00   (40.81610072727273, -73.90402954545455)  american
5   2014-07-09  19:01:00                  (40.815306, -73.8987435)  american
..         ...       ...                                       ...       ...
312 2014-09-28  07:18:00   (40.87678204081632, -73.85872673469387)  prestige
313 2014-09-28  09:13:00               (40.8768236875, -73.853798)  prestige
314 2014-09-29  08:48:00                 (40.8350854, -73.9118444)  prestige
317 2014-09-29  17:00:00  (40.859985040816326, -73.84296128571428)  prestige
319 2014-09-30  23:39:00  (40.848002300000005, -73.84378242145698)  prestige

[743 rows x 4 columns]


In [30]:
big_df.to_csv("small_data.csv")

In [9]:
def get_uber_df(carrier):  
    filename = "raw_data/" + carrier + ".csv"
    print(filename)
    n = sum(1 for line in open(filename)) - 1
    s = int(n / 1000)
    print(s)
    skip = sorted(random.sample(range(1, n+1), n-s))
    df = pd.read_csv(filename, skiprows=skip)
    df["date"] = pd.to_datetime(df["Date/Time"]).dt.date
    df["time"] = pd.to_datetime(df["Date/Time"]).dt.time
    df["location"] = list(zip(df.Lat, df.Lon))
    df["carrier"] = "uber"
    df = df[["date", "time", "location", "carrier"]]
    return df

In [14]:
big_df = pd.DataFrame()
for carrier in ["Uber_Jul", "Uber_Aug", "Uber_Sep"]:
    big_df = big_df.append(get_uber_df(carrier))

raw_data/Uber_Jul.csv
796
raw_data/Uber_Aug.csv
829
raw_data/Uber_Sep.csv
1028


In [15]:
print(big_df)

            date      time                       location carrier
0     2014-07-01  16:20:00            (40.7189, -74.0026)    uber
1     2014-07-02  18:20:00            (40.7713, -73.9836)    uber
2     2014-07-04  11:52:00            (40.7739, -73.8724)    uber
3     2014-07-04  18:17:00            (40.7527, -73.9675)    uber
4     2014-07-05  12:31:00            (40.7726, -73.9812)    uber
...          ...       ...                            ...     ...
1023  2014-09-30  08:28:00            (40.7843, -73.9585)    uber
1024  2014-09-30  13:01:00            (40.7287, -73.9548)    uber
1025  2014-09-30  15:00:00  (40.7764, -73.94800000000001)    uber
1026  2014-09-30  16:12:00            (40.7758, -73.9507)    uber
1027  2014-09-30  19:30:00            (40.7587, -73.9775)    uber

[2653 rows x 4 columns]


In [16]:
big_df = pd.read_csv("small_data.csv").append(big_df)

In [17]:
big_df.to_csv("small_data.csv")